<a href="https://colab.research.google.com/github/ammobam/Display_SensorData/blob/main/210731_%EB%8D%B0%EC%9D%B4%ED%84%B0_%ED%83%90%EC%83%89(4)%EC%BB%AC%EB%9F%BC_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **디스플레이 생산라인 센서 데이터 분석

## 주제 : 불량품에 영향을 주는 요인 찾기
- 결과 : "ㅇㅇ센서에서 ㅁㅁ면 불량품"
1. 주요 요인 피처 5개 추출
2. 불량품/양품 구분하는 모델도 찾아야 함


- 머신러닝 이용
---

## 컬럼이름 분류하기
- 데이터는 센서 데이터로 약  800여개의 컬럼이 있음
- 이름이 비슷한 컬럼이 다수 있으며 생산라인이 Left, Right으로 나뉘어 동일한 공정을 수행중임
- 컬럼이름을 기준으로 컬럼을 분류하고자 함
    - LEFT
    - RIGHT
    - 공통
- 컬럼을 분류하면 피처 수를 줄여 데이터탐색이 가능함
    - 이름이 유사한 컬럼간 종속성을 확인
    - LEFT, RIGHT 각 공정에 따라 레이블에 미치는 영향 확인 


In [ ]:
# 데이터 불러오기
import pandas as pd
df_origin = pd.read_csv('/content/drive/MyDrive/나무플래닛/1. 데이터_디스플레이/factory_glass_2016.zip', encoding='cp949')

In [ ]:
# 컬럼이름 리스트 저장하기
col = df_origin.columns.tolist()
#col

### 1. 앞 3글자가 동일한 컬럼을 분류하고 확인


In [26]:
# 앞글자 3자리가 같은 컬럼의 집단 개수 확인
col_name_3=[]
for name in col:
    name = str(name)[:3]
    col_name_3.append(name)

print(len(set(col_name_3)))  # 61개

61


In [ ]:
# 앞글자 3자리가 key, 이름이 유사한 컬럼이 value인 딕셔너리 생성
# 빈 딕셔너리 생성
col_name_3={}
# 인덱스, 컬럼이름에 대해 반복
for i, key in enumerate(col):
    key = str(key)[:3]
    # 만약 key가 딕셔너리에 없으면 키, 인덱스 생성
    if key not in col_name_3:
        col_name_3[key] = [col[i]]
        ## 딕셔너리에 넣을 때 int가 아닌 list로 넣어야 함

    # 만약 key가 딕셔너리에 있으면 인덱스 추가
    else:
        col_name_3[key].append(col[i])

print(len(col_name_3)) # 컬럼집단 61개 잘 만들어짐
col_name_3

In [85]:
# 2. 컬럼명에 LEFT, RIGHT가 있으면 분류

# left, right 분류하여 저장할 딕셔너리 생성
dict_l_r = {'Left':[],
            'Right':[],
            'Center':[]}

# 각 key에 대해 반복 수행
for key in col_name_3:
    #print(col_name_3[key])
    #print(len(col_name_3[key]))


    for i in range(len(col_name_3[key])):
        #print(str(col_name_3[key][i]).lower())

        # left라는 단어가 있으면 
        if ('left' or '\_l' or '\.l\.') in str(col_name_3[key][i].lower()):
            dict_l_r['Left'].append(col_name_3[key][i])
        
        # right라는 단어가 있으면
        elif ('right') in str(col_name_3[key][i].lower()):
            dict_l_r['Right'].append(col_name_3[key][i])

        # left, right가 아니면
        else:
            dict_l_r['Center'].append(col_name_3[key][i])

    # 각 컬럼이름에 대해 반복 수행
        #print(str(col_name_3[key]).lower())

# 3. 컬럼명에 _L, _R이 있으면 분류
    # _L, _R 포함한 이름들 먼저 확인해보기

dict_l_r
for key in dict_l_r:
    print(key, ":", len(dict_l_r[key]))

Left : 27
Right : 24
Center : 790


In [74]:
import re
help(re.finditer)

Help on function finditer in module re:

finditer(pattern, string, flags=0)
    Return an iterator over all non-overlapping matches in the
    string.  For each match, the iterator returns a Match object.
    
    Empty matches are included in the result.



In [ ]:
col_name_3

In [88]:
for key in col_name_3:
    for i in range(len(col_name_3[key])):
        # 글자에 '.r.'가 들어가면 출력
        # re.메서드라서 .이 메타문자가 아닌 문자로 읽히려면 \. 로 작성해야 함
        st = [m.start() for m in re.finditer('\.r\.', str(col_name_3[key][i].lower()))]
        print(st)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[